# Задание 1

## Классификация текстов: спам-фильтр для SMS

In [73]:
import numpy as np

Считаем датасет в Python.

In [74]:
data = np.genfromtxt('../SMSSpamCollection.txt', dtype=None, delimiter='\t')

Подготовим два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. Метка 1 соответствует спаму, метка 0 - "не спаму".

In [99]:
textdata = [[0 if elem[0].decode('utf8') == 'ham' else 1, elem[1].decode('utf8')] for elem in data]
splitdata = [list(l) for l in zip(*textdata)]
target, texts = splitdata

Получим матрицу признаков X при помощи CountVectorizer из библиотеки sklearn.

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
sfecv = CountVectorizer()
X = sfecv.fit_transform(texts)

Используем логистическую регрессию для классификации данного набора текстов.

In [101]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=241)

Оценим качество классификации при помощи кросс-валидации с 10 фолдами.

In [102]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lrc, X, target, scoring='f1', cv=10)
print('Mean score:', scores.mean())

Mean score: 0.933419572279


Обучим классификатор на всей выборке и спрогнозируем с его помощью класс для следующих сообщений:

In [103]:
messages = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?", 
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
    "Only 99$"
]
lrc.fit(X, target)
res = lrc.predict(sfecv.transform(messages))
print(res)

[1 1 0 0 0]


Исследуем качество классификации при помощи линейного классификатора при использовании биграмм, триграмм и n-грамм при n=[1,3].

In [104]:
ngrammcombs = [(1,1), (2,2), (3,3), (1,3)]
LR_ngscores = []
for ng_range in ngrammcombs:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(texts)
    LR_ngscores.append(cross_val_score(lrc, X, target, scoring='f1', cv=10).mean())

# Ответ
print([float("{0:0.2f}".format(i)) for i in LR_ngscores])

[0.93, 0.82, 0.72, 0.92]


Используем наивный Байес вместо логистической регрессии.

In [105]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

NB_ngscores = []
for ng_range in ngrammcombs:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(texts)
    NB_ngscores.append(cross_val_score(nbc, X, target, scoring='f1', cv=10).mean())

# Ответ
print([float("{0:0.2f}".format(i)) for i in NB_ngscores])

[0.93, 0.64, 0.38, 0.89]


Используем в качестве признаков в логистической регрессии Tf*idf из Tfidfvectorizer на униграммах.

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(ngram_range=(1,1))
X = tfidfv.fit_transform(texts)
print('Mean score: {0:0.2f}.'.format(cross_val_score(lrc, X, target, scoring='f1', cv=10).mean()))

Mean score: 0.85.


Качество на кросс-валидации понизилось (0.85 < 0.93) по сравнению с CountVectorizer() на униграммах.

### Попробуем повысить качество на кросс-валидации.

1) В линейной модели LogisticRegression() по умолчанию используется L2-регуляризация. Можно попробовать использовать L1-регуляризацию для отбора признаков, а также осуществить поиск оптимальных параметров для коэффициента регуляризации. Это может несколько повысить качество классификации.

Используем результат CountVectorizer() в качестве признаков для логистической регрессии.

In [110]:
from sklearn.model_selection import GridSearchCV

X = sfecv.fit_transform(texts)
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4, 1e6]
    }
gscv = GridSearchCV(lrc, param_grid, scoring='f1', cv=10)
gscv.fit(X,target)
print(gscv.best_params_)
print(gscv.best_score_)

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 d

{'C': 100.0, 'penalty': 'l2'}
0.944148253522


In [109]:
res = gscv.best_estimator_.predict(sfecv.transform(messages))
print(res)

[1 1 0 0 0]


По результатам GridSearchCV можно сделать вывод, что первоначальная регуляризация в линейной модели были чересчур строга, поскольку увеличение коэффициента C (уменьшение коэффициента регуляризации) привело к повышению качества на кросс-валидации.

2) Проверим, является ли данная выборка сбалансированной.

In [112]:
print(len(target), sum(target))

5574 747


## Выводы

In [113]:
# TODO: Finish!